In [61]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## The forward and backward passes

Let's start by importing nb_01. And I just copied and pasted the three lines we used to grab the data, and I am just going to pop them into a fucntion, so we can use it to grab MNIST when we needed.
And now that we know about broadcasting let's create a normalization function that takes our tensor x and subtracts the mean and divides it by the standard deviation.

In [62]:
#export
from exp.nb_01 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

In [63]:
x_train,y_train,x_valid,y_valid = get_data()

In [64]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.1304), tensor(0.3073))

The mean and standard deviation are not 0 and 1, but we want them to be. So that means that we need to subtract the mean and divide by the standard deviation. 

But not for the validation set! We do not subtract the validation's set mean and divide it by the standard validation's set standard deviation. Because if we did those two data sets would be on a totally different scale. So if the training set was mainly green frogs, and the validation set was mainly red frogs, then if we normalize with the validation set mean variance, we would end up witht hem both having the same coloration and we would not be able to tell the two them apart. 

So that is an important thing to remember, when normalizing, is to **always make sure your validation and training set are normalized in the same way.**

In [65]:
x_train = normalize(x_train, train_mean, train_std)

# NB: Use TRAINING, not validation, mean for validation set
x_valid = normalize(x_valid, train_mean, train_std)

In [66]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(-7.6999e-06), tensor(1.))

So after doing that our mean is pretty close to zero and our standard deviation is very close to 1. And it would be nice to have somethign to easily check that these are tru so let's create a `test_near_zero` function. 

And then test that the mean is near zero, and the one minus the standard deviation is near zero.

In [67]:
#export
def test_near_zero(a,tol=1e-3): assert a.abs()<tol, f"Near zero: {a}"

In [68]:
test_near_zero(x_train.mean())
test_near_zero(1-x_train.std())

Let's define and n and m and c. The same as before, so n,m the size of the training set, c the number of activations we eventually need in our model.

In [69]:
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(50000, 784, tensor(10))

## Foundations version

### Basic architecture

Our model is going to have one hidden layer. Normally we would want the final output to have 10 activations, because we would use cross entropy across those 10 activations. But to simplify things for now, we are not going to use cross entropy but mean squared error, which means we are going to have 1 activations. (Which makes no sense from a modelling point of view, but we will fix that later and but just to simplify things for now)

So let's create a simple neural net with a single hidden layer and a single output activation which we are going to use, mean squared error.
So let's pick a hidden size, so the number of hidden we will make 50

In [70]:
# num hidden
nh = 50

So we need two weight matrices and two bias vectors. They are normal random numbers of size m, the number of columns, 768, by nh, number of hidden. And then w2 is nh by 1.

Our inputs of the first layer are mean zero standard deviation of one. We want the inputs of the second layer to also be mean zero standard deviation of one. Well, how are we going to do that? Because if we just grab some normal random numbers, and then we define a function called `lin()` (see below), this is our linear layer, which is `x@w + b`, and then create `t`, which is the activation of the linear layer, with our validation set and our weights and biases, we have a mean of minus 5 and standard deviation of 27. Which is terrible.

In [71]:
# simplified kaiming init / he init
w1 = torch.randn(m,nh)/math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)/math.sqrt(nh)
b2 = torch.zeros(1)

In [72]:
test_near_zero(w1.mean())
test_near_zero(w1.std()-1/math.sqrt(m))

In [73]:
# This should be ~ (0,1) (mean,std)...
x_valid.mean(),x_valid.std()

(tensor(-0.0059), tensor(0.9924))

In [74]:
def lin(x, w, b): return x@w + b

In [75]:
t = lin(x_valid, w1, b1)

In [76]:
#...so should this, because we used kaiming init, which is designed to do this
t.mean(),t.std()

(tensor(-0.0059), tensor(0.9562))

So I am going to let you work through this at home, but once you look at what actually happens when you multiply those things together and add them up as you do in matrix multiplication, you will see that you will not end up with zero and one. but if you instead divide by square root m (root 768) then it is actually damn good. 

So this is something which PyTorch calls Kaiming initialization. Named after Kaiming He, who wrote the paper, or was the lead writer of a paper we will look at in a moment. So the weights randn gives you random numbers with a mean of zero and a standard deviation of 1. So if you divide by root m, it will have a mean of zero and a standard deviation of 1 on root m.

So in general normal random numbers of mean zero and standard deviation of 1 over root of whatever m/nh will give you an output of 1,0. So this may seem like a minor issue, but as we will see in the next couple of lessons, it is the thing the matters when it comes to training neural nets. It is actually in the last few monthspeople have really been noticing how important this is.

There have been things like Fixup Initialization, where these folks actually trained a 10000 layer neural network, with no normalization layers, just by basically doing careful initialization. So people are really spending a lot of time now thinking ok: how we initialize things is really important. And we had a lot of success with things like one cycle training and superconvergence, which is all about what happens in those first few iterations, and it turns out that it is all about initializations. So we will spend a lot of time studying this in depth.

So the first thing that I am going to point out, def `lin(x, w, b): return x@w + b` is actually not how our first layer is defined. Our first layer is actually defined like this: `t = relu(lin(x_valid, w1, b1))`. 

So let's define ReLU. So ReLU is just grab our data and replace any negatives with zero, that is what clamp_min is.

In [77]:
def relu(x): return x.clamp_min(0.)

In [78]:
t = relu(lin(x_valid, w1, b1))

So there are a lot of things I could have written this, but if you can do it with something that is a single function in PyTorch, it is almost always faster, because that thing is generally written in C for you. So try to find the thing that is close as to what you want as possible, there's a lot of functions in PyTorch. So that's a good thing of implementing ReLU. And unfortunately, that does not have a mean of zero and standard deviation of 1. Why not?

In [79]:
#...actually it really should be this!
t.mean(),t.std()

(tensor(0.3770), tensor(0.5577))

From pytorch docs: `a: the negative slope of the rectifier used after this layer (0 for ReLU by default)`

$$\text{std} = \sqrt{\frac{2}{(1 + a^2) \times \text{fan_in}}}$$

This was introduced in the paper that described the Imagenet-winning approach from *He et al*: [Delving Deep into Rectifiers](https://arxiv.org/abs/1502.01852), which was also the first paper that claimed "super-human performance" on Imagenet (and, most importantly, it introduced resnets!)

Read section 2.2 of the paper for the full explanation, but if you multiply by the sqrt of 2/m, it gets you much closer.

In [80]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2/m)

In [81]:
w1.mean(),w1.std()

(tensor(0.0005), tensor(0.0506))

In [82]:
t = relu(lin(x_valid, w1, b1))
t.mean(),t.std()

(tensor(0.4396), tensor(0.7536))

It doesn't give us a very nice mean though, naturally our mean is now half, not zero.

I haven't seen anybody talk about this in the literature, but something I was just trying over the last week, is something kind of obvious which is to replace ReLU not just with `x.clamp_min()` but with `x.clamp_min(0.) - 0.5`

And in my brief experiments that seems to help. So there's another thing that you can try out and see if it actually helps or if I'm just imagining things. It certainly returns you to the correct mean.

So now that we have this formula `w1 = torch.randn(m,nh)*math.sqrt(2/m)`, we can replace it with `init.kaiming_normal()` , because it is the same thing.
And let's check that it does the same thing.

In [83]:
#export
from torch.nn import init

In [84]:
w1 = torch.zeros(m,nh)
init.kaiming_normal_(w1, mode='fan_out')
t = relu(lin(x_valid, w1, b1))

In [85]:
w1.mean(),w1.std()

(tensor(-0.0002), tensor(0.0502))

In [86]:
t.mean(),t.std()

(tensor(0.5469), tensor(0.8255))

In [87]:
w1.shape

torch.Size([784, 50])

And it does

In [88]:
# init.kaiming_normal_??

`fan_in` preserves the magnitude of the variance n the forward pass.<br>
`fan_out` preserves the magnitude of the variance in the backward pass.

Basically all it is saying is, are you dividing by root m or root nh.<br> 
Because if you divide by root m, as you will see in that part of the paper I suggested you read, that will keep the variance at one during the forward pass, but if you use nh, it will give you the right variance in the backward pass.

So it is weird that I had to say fan_out because according to the documentation that is for the backward pass to keep the unit variance. So why did I need that?



In [89]:
import torch.nn

In [90]:
torch.nn.Linear(m,nh).weight.shape

torch.Size([50, 784])

Well, it is because our weight shape is 784 by 50, but if you actually create a linear layer with PyTorch, of the same dimensions, it creates it of 50 by 784, the opposite. So how can that possibly work? These are the kind of things that is useful to dig into, so how is this working?

To find out how it is working, you need to look at the source code. So you can either set up VS Code and do that, or you can do it with ?? 

There's a forward function that calls something called F.linear. In PyTorch, F always refers to the torch.nn.functional module. Because it is used everywhere, they've decided that that is worth a single letter.
So torch.nn.functional.linear is what it calls, and let's look at how that is defined:
inputmatmul(weight.t()). t means transpose. So now we know in PyTorch a linear layer doesn't just do a matrix product, it does a matrix product with a transpose. So in other words, it is going to turn this into 785 by 50, and then do it. So that is why we had to give it the opposite information when we were trying to do it with our linear layer, which doesn't have transpose.

In [91]:
# torch.nn.Linear.forward??

In [92]:
# torch.nn.functional.linear??

So the main reason I'm showing you this is to show you how can dig into the PyTorch source code, see exactly what is going on, so when you come across these kid of questions, you want to be able to answer them yourself.

Which also then leads to the question: if this is how linear layers can be initialized, what about convolutional layers?

What does PyTorch do to convolutional layers? So we can look at torch.nn.Conv2d. 

In [93]:
# torch.nn.Conv2d??

It basically doesn't have any code, just documentation, all of the code gets passed down to something called _ConvNd.
And so you need to know how to find these things, so if you go the the bottom, you can find the file name it is in, and so you can see this is actually torch.nn.modules.conv, so you can find torch.nn.modules.conv._ConvNd. 

So here it is, and how it initializes thing, and it calls `kaiming_uniform`, which is basically the same as kaiming_normal but uniform instead, but it has a special multiplier of `math.sqrt(5)`.

And that is not documented anywhere, I have no idea where it comes from and from my experiments, `init.kaiming_uniform_(self.weight, a=math.sqrt(5))` seems to work pretty badly. As you will see.

So it is kind of useful to look inside the code so when you write your own code, presumably someone has put math.sqrt(5) there for a reason. Wouldn't it have been nice if they had a url above it with a link to the paper that they're implementing, so that we could see what is going on, so that the next person around could see what the hell you are doing.
So this particular thing I have a strong feeling isn't great, as you will see.

In [94]:
# torch.nn.modules.conv._ConvNd.reset_parameters??

So we are going to try this thing that is subtracting .5 from our ReLU, so like this is prety cool right, we've already designed our new activation function. Is it great, terrible, I don't know, but it is this kind of level of tweak - when people write papers, this is the normal level, it is a minor change to one line of code, it would be interesting to see how much it helps. But if I use it, you can see

In [95]:
# what if...?
def relu(x): return x.clamp_min(0.) - 0.5

In [96]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2./m )
t1 = relu(lin(x_valid, w1, b1))
t1.mean(),t1.std()

(tensor(0.0236), tensor(0.8058))

Now I have a mean of 0. And interstingly it helps the variance as well, before it was generally around .7 - .8, now it is generally above .8. So it helps both, which makes sence as to why I think I see these better results.
So now we have ReLU, we have a linear, we have init, so we can do a forward pass.
And so here it is. 

So remember in PyTorch, a model can just be a function. And so here's our model, just a function that does one linear layer, one linear layer, one relu and one more linear layer.

In [97]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [98]:
%timeit -n 10 _=model(x_valid)

4.01 ms ± 55.1 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Add an assert, to make sure the shape seems sensible.

In [99]:
assert model(x_valid).shape==torch.Size([x_valid.shape[0],1])

### Loss function: MSE

So the next thing we need for our forward pass is a loss function, and as I said, we are going to simplify things for now by using MSE. Even although that a dump idea obviously. Our model is returning something of size 10000 by 1.

In [100]:
model(x_valid).shape

torch.Size([10000, 1])

We need `squeeze()` to get rid of that trailing (,1), in order to use `mse` with a vector.<br> 
(Of course, `mse` is not a suitable loss function for multi-class classification; we'll use a better loss function soon. We'll use `mse` for now to keep things simple.)

In [101]:
#export
def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean()

For mse we have to make sure they are floats, so let's convert them.

In [102]:
y_train,y_valid = y_train.float(),y_valid.float()

In [103]:
preds = model(x_train)

In [104]:
preds.shape

torch.Size([50000, 1])

In [105]:
mse(preds, y_train)

tensor(29.8964)

#### Let's see what each line in the forward pass returns

In [106]:
x_train.shape

torch.Size([50000, 784])

In [107]:
l1 = lin(x_train, w1, b1); l1, l1.shape

(tensor([[-1.7458, -1.4146,  1.0774,  ..., -1.5136, -1.5683, -0.4717],
         [ 0.4722, -1.0813, -0.8704,  ..., -0.8512, -1.8047, -0.9888],
         [-0.4605, -0.6725,  1.4122,  ...,  2.1583,  0.0054, -1.6397],
         ...,
         [-2.5006,  1.6314, -0.1732,  ..., -0.8130, -0.9691,  0.1910],
         [ 0.2092, -1.0262,  0.8989,  ..., -1.4888, -1.5526, -1.3232],
         [-1.5572, -0.7802,  1.4053,  ...,  0.4839, -0.6478, -0.3678]]),
 torch.Size([50000, 50]))

In [108]:
l2 = relu(l1); l2, l2.shape

(tensor([[-0.5000, -0.5000,  0.5774,  ..., -0.5000, -0.5000, -0.5000],
         [-0.0278, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
         [-0.5000, -0.5000,  0.9122,  ...,  1.6583, -0.4946, -0.5000],
         ...,
         [-0.5000,  1.1314, -0.5000,  ..., -0.5000, -0.5000, -0.3090],
         [-0.2908, -0.5000,  0.3989,  ..., -0.5000, -0.5000, -0.5000],
         [-0.5000, -0.5000,  0.9053,  ..., -0.0161, -0.5000, -0.5000]]),
 torch.Size([50000, 50]))

In [109]:
out = lin(l2, w2, b2); out, out.shape

(tensor([[-0.5796],
         [ 0.0443],
         [ 0.1886],
         ...,
         [-0.4149],
         [ 0.5629],
         [ 0.1317]]), torch.Size([50000, 1]))

In [110]:
loss = mse(out, y_train); loss, loss.shape

(tensor(29.8964), torch.Size([]))

In [111]:
y_train.shape

torch.Size([50000])

#### There it is, we've done a forward pass.

A forward pass is useless, what we need is a backward pass, because that's the thing that tells us how to update our parameters.


So we need gradients. Ok. How much do you want to know about matrix calculus? I don't know, it's up to you. But if you want to know everything about matrix calculus, I can point you to this excellent paper by Terence Parr and Jeremy Howard, which tells you everything about matrix calculus from scratch.

https://arxiv.org/abs/1802.01528

So this is a few weeks work to get through, but it absolutely asumes nothing at all. Basically Terrence and I felt like: we don't know any of this stuff, let's learn all of it and tell other people. So we wrote it with that in mind, and so this will take you all the way up to know everything that you need for deep learning. You can actually get away with a lot less, but if you're hear, maybe it is worth it. I tell you what you do need to know. 

**What you do need to know is the chain rule.**

We start with some input and we stick it through the first linear layer<br> 
and then through ReLU<br> 
and then through the second linear layer<br> 
and then through mse<br> 
and that gives us our predictions. 

Or, to put it another way:<br> 
We start with x<br> 
Put it through the function lin1().<br> 
Take the output of that and we put it through the function relu().<br> 
Take the output of that through a function lin2()<br> 
Take the output of that and put it through a function mse().<br> 
And strictly speaking mse has a second argument, the actual target value.

`y_hat = mse(lin2(relu(lin1(x))), y)`

So if we simplify things down a bit, we could just say

`y = f(u), u = f(x)`  that is like a function of a function.
And the derivative is `dy/dx = dy/du * du/dx` That's the chain rule.

### Gradients and backward pass

We will now calculate the gradients and store them in the `.g` attribute

![fb pass](images/FB_pass.jpg)

So to do the chain rule, we are going to have to start with the very last function, the loss function, and get the gradient of the loss with respect to output of previous layer.<br> 
For the output of the previous layer, the MSE is just input minus target squared, so the derivative of that is just 2 times input minus target, because the derivative of bla squared is two times bla. So that's it.

I need to store that gradient somewhere. For the chain rule I need to multiply all these things together. So I store it inside a `.g` attribute of the previous layer. So the input of MSE is the same as the output of the previous layer. And now we can easily refer to it later.

In [112]:
def mse_grad(inp, targ): 
    """
    Calculate gradient of the loss with respect to the output of previous
    layer, and store it in the 'g' attribute of inp.
    
    Args:
        inp (rank 2 tensor): output of final linear layer, input to mse funct,
                            shape=(50000, 1)(n, num_out)
        targ (rank 1 tensor): y labels, shape=(50000)(n)
    """
    
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]

ReLU gradient is just the input greather than zero.<br>
We need to multiply this by the gradient of the next layer, which remember, we stored away (out.g), so we can just grab it.

In [113]:
def relu_grad(inp, out):
    """
    Calculate gradient of relu with respect to the input activations 
    and store it in the 'g' attribute of inp.
    
    Args:
        inp (rank 2 tensor): output of layer 1 (which was fed into initial relu funciton), 
                            shape=(n, nh)
        out (rank 2 tensor): output after relu function, shape=(n, nh)
    """

    inp.g = (inp>0).float() * out.g

Same thing for the linear layer. 

This seems like an insightful shortcut...
#### The gradient of a matrix product is the matrix product with the transpose:
`inp.g = out.g @ w.t()`

In [114]:
def lin_grad(inp, out, w, b):
    """
    Caluculate the gradients of matrix multiplication with respect to the input
    and store them respectively 
    
    Args (for first layer):
        inp (rank 2 tensor): input to linear layer 1 (x_train), shape=(n, m) (50000, 784)
        out (rank 2 tensor): output of linear layer 1, shape=(n, nh) (50000, 50)
        w (rank 2 tensor): weights, shape=(m, nh) (784, 50)
        b (rank 1 tensor): bias, shape=(nh) (50)
    """
    
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

Here's a function that does the forward pass that we have already seen.

And then it goes backward, it calls each of the gradients backwards, in reverse order, because we know we need that for the chain rule.
And you can notice that every time we are passing in the result of the forward pass. And it also has access to the gradient of the next layer.
This is called backpropagation. So when people say 'backpropagation is not just the chain rule', they are basically lying to you. Backrpropagation is the chain rule, where we just save away all the intermediate calculations, so we don't have to calculate them again.
So this is a full forward and backward pass.

In [115]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    # we don't actually need the loss in backward!
    loss = mse(out, targ)
    
    # backward pass:
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

One interesting thing here is this value `loss = mse(out, targ)`, we never actually use it. Because the loss never actually appears in the gradients, I mean, by the weights. You still probably want it to print it out or whatever, but it does not appear in the gradients.

In [116]:
forward_and_backward(x_train, y_train)

In [117]:
# Save for testing against later
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig  = x_train.g.clone()

So w1g, w2g now contain all of our gradients, which we are going to use for the optimizer. So let's cheat and use PyTorch autograd to check our results, because PyTorch can do it for us. Let's clone all of our weights and biases. And input. and then turn on requires_grad_() for all of them.

In [118]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

`requires_grad_` is how you take a PyTorch tensor and then turn it into a magical auto gradified tensor. So what it is now going to do is everything that is calculated with test tensor is basically going to keep track of what happened, so it keeps track of the first teps in the forward_and_backward, so it can do the backward. You can write it yourself, you just need to make sure that each time you do an operation, you remember what it is and you can go back through them in reverse order. 

So now we have done requires_grad_ we can do the forward pass like so, and gives us our loss.

In [119]:
def forward(inp, targ):
    # forward pass:
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    # we don't actually need the loss in backward!
    return mse(out, targ)

In [120]:
loss = forward(xt2, y_train)

In [121]:
loss.backward()

In [122]:
test_near(w22.grad, w2g)
test_near(b22.grad, b2g)
test_near(w12.grad, w1g)
test_near(b12.grad, b1g)
test_near(xt2.grad, ig )

And all of our gradients were correct. So that is pretty interesting, that is an actual neural network, that contains all of the main pieces we are going to need, and we have written all these pieces from scratch. 

## Refactor model

Let's do some refactoring, and this is massively, closely stolen from the PyTorch api. And it is intersting, I didn't have it in mind, but when I kept refactoring, I just noticed: i recreated the PyTorch api.

Let's take each of our layers, `Relu()` and `Lin()`, and create classes. 

For the forward, use `__call__`. This means that we can now treat this as a function. So if you call this `class Relu()` as a function, it calls this function `__call__`.

### Layers as classes

In [123]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)-0.5
        return self.out
    
    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g

Let's save the input, calculate and save the output, and return the output.

And then for backward, we will calculate the gradient and save it inside `self.inp.g`.<br> 

So it is the same as earlier, but the forward and backward are moved into the same class.

For a more concrete example, using the earlier model we created:<br>
`Relu` would take `l1` as input, and return `l2` as output for the forward pass.<br> 
When `backward` is called, `l2` will already have a gradient inside `.g`, and it will be used to calculate `l1.g`

In [124]:
class Lin():
    def __init__(self, w, b): self.w,self.b = w,b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp@self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        # Creating a giant outer product, just to sum it, is inefficient!
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

One thing to notice, the backward pass here in gradient, for linear we don't just want the gradient of the outputs with respect to the inputs `self.inp.g`, we also need the gradient of the output with respect to the weights `self.w.g`, and the output with respect to the biases `self.b.g`.<br>
So there's our linear layer, forward and backward.

And then we have our `Mse()`. In the forward, we save the `inp` and `targ` for later, calculate the `out`.

And the `backward` is the same.

In [125]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

So with this refactoring, we can create our `Model` class with a list of all of our layers.

In [126]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

Let's define `loss = Mse()` , and `__call__`, which goes through all of our layers, and executes `x = l(x)`. 

* That is the function composition, we just call the function on the result of the previous thing.

And then at the end call `self.loss`, and then for `backward` we do the exact opposite. We do `self.loss.backward()` and then we do the reversed layers and call `backward` on each one.

And remember the backward passes are going to save the gradient away in `.g` in each respective class instance.

We can then create our model and call it, call backward and check that the gradients are correct.

In [127]:
w1.g,b1.g,w2.g,b2.g = [None]*4    # zero out the gradients first

model = Model(w1, b1, w2, b2)

In [128]:
%time loss = model(x_train, y_train)

CPU times: user 80 ms, sys: 8 ms, total: 88 ms
Wall time: 21.6 ms


In [129]:
%time model.backward()

CPU times: user 3.76 s, sys: 5.54 s, total: 9.3 s
Wall time: 2.47 s


In [130]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

Our gradients are correct, but it took a long time. 

### Module.forward()

There's a lot of duplicate code, let's get rid of it. 

Create a new `Module()` class, which handles the `inp` and `out`.<br> 
So now we are not going to use `__call__` to implement our forward, but call something called `forward` which we will initially set to `raise Exception('not implemented')`.

In [119]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)

In [120]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)-0.5
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [121]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = torch.einsum("bi,bj->ij", inp, out.g)
        self.b.g = out.g.sum(0)

Where we calculated the derivative of the output of the linear layer with respect to the weights, where we did an unsqueeze and an unsqueeze, we can re-express that with einsum. 

`self.w.g = torch.einsum("bi,bj->ij", inp, out.g)`

In [122]:
class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

In [123]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

So the code is now neater and executes faster.

In [124]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [125]:
%time loss = model(x_train, y_train)

CPU times: user 80 ms, sys: 8 ms, total: 88 ms
Wall time: 21.9 ms


In [126]:
%time model.backward()

CPU times: user 276 ms, sys: 208 ms, total: 484 ms
Wall time: 132 ms


In [127]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

And now this almost looks like PyTorch - and we can see why, why we have to inherit from nn.Module, why we have to define forward, it let's PyTorch factor out all the duplicate stuff. So all we have to do is do the implementation.

### Without einsum

And then once we realised, einsum is exactly the same as inp.t() @ out.g, so we replaced einsum with a matrix product, and that is even faster. 

In [128]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = inp.t() @ out.g
        self.b.g = out.g.sum(0)

In [129]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [130]:
%time loss = model(x_train, y_train)

CPU times: user 96 ms, sys: 32 ms, total: 128 ms
Wall time: 32.8 ms


In [131]:
%time model.backward()

CPU times: user 256 ms, sys: 144 ms, total: 400 ms
Wall time: 98.9 ms


In [132]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

So now we have basically implemented nn.Linear and nn.Module, so we can use it. And the forward is almost the same speed and the backward pass is about twice as fast, I guess because we are calculating all of the gradients, and they are not calculating all of them, only the ones they need, but it is basically the same thing.

### nn.Linear and nn.Module

In [133]:
#export
from torch import nn

In [134]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out)]
        self.loss = mse
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x.squeeze(), targ)

In [135]:
model = Model(m, nh, 1)

In [136]:
%time loss = model(x_train, y_train)

CPU times: user 124 ms, sys: 40 ms, total: 164 ms
Wall time: 87.6 ms


In [137]:
%time loss.backward()

CPU times: user 96 ms, sys: 64 ms, total: 160 ms
Wall time: 95.2 ms


So at this point we are ready in the next lesson to do a training loop.
We have a multilayer fully connected neural network, a rectified network, matrix multiply organised, forward and backward passes nicely refactored out into classes and a module class. So in the next lesson we will see how far we can get, hopefully we will build a high quality fast resnet. And we will also take a very deep dive in optimizers and callbacks and training loops and normalization methods.

## Export

In [138]:
!python notebook2script.py 02_fully_connected.ipynb

Converted 02_fully_connected.ipynb to exp/nb_02.py
